In [1]:
#1)Добавим нужные библиотеки
import re
import pyshark
import nest_asyncio
nest_asyncio.apply()
import json

In [2]:
#2)Прочитаем пакеты из трейса и применим фильтр по протоколам sctp и s1ap
capture = pyshark.FileCapture('./s1_gtv2_dia.pcapng')

In [3]:
#3)Для более детального изучения данный блок позволяет получить все поля пакета по его номеру. 
for i in capture:
    if i.number == '5':
        print(i.highest_layer)
        print(i.diameter._all_fields)
        print(dir(i.diameter))
        print(i.diameter.cmd_code)
        print(i.diameter.avp_code)
        print(i.diameter.session_id)
        

DIAMETER
{'diameter.version': '0x01', 'diameter.length': '796', 'diameter.flags': '0x40', 'diameter.flags.request': 'False', 'diameter.flags.proxyable': 'True', 'diameter.flags.error': 'False', 'diameter.flags.T': 'False', 'diameter.flags.reserved4': 'False', 'diameter.flags.reserved5': 'False', 'diameter.flags.reserved6': 'False', 'diameter.flags.reserved7': 'False', 'diameter.cmd.code': '318', 'diameter.applicationId': '16777251', 'diameter.hopbyhopid': '0x6dbd2fb7', 'diameter.endtoendid': '0x1d43930d', 'diameter.answer_to': '4', 'diameter.resp_time': '0.002483000', 'diameter.avp': '00:00:01:07:40:00:00:2c:65:70:63:2e:67:65:6e:65:77:2e:63:6f:6d:3b:30:30:30:32:36:38:37:35:36:36:3b:30:30:30:30:33:3b:30:30:30:30:30', 'diameter.avp.code': '263', 'diameter.avp.flags': '0x40', 'diameter.flags.vendorspecific': 'False', 'diameter.flags.mandatory': 'True', 'diameter.avp.flags.protected': 'False', 'diameter.avp.flags.reserved3': 'False', 'diameter.avp.flags.reserved4': 'False', 'diameter.avp.f

In [23]:
# Attach request 
att_att_list = []
att_com_list = []
for packet in capture:
    if (packet.highest_layer == 'S1AP'):
        d = packet.s1ap._all_fields
        #Находим все пакеты с кодом процедуры DownlinkNASTransport
        if (packet.s1ap.procedurecode == '12'):
            #Получаем идентификатор транспортной сессии SCTP
            get_value = packet.sctp.data_sid
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if packet.sctp.data_sid == get_value:
                #Преобразуем пакет в словарь, затем в список
                get_dict = dict(packet.s1ap._all_fields)
                keys = list(get_dict.items())
                #Делаем перебор всех значений в списке, чтобы найти пакеты, содержащие сообщение Authentication Request
                for i in keys:
                    if (i == ('nas-eps.nas_msg_emm_type', '0x41')):
                        print("Attach Request was detected in ", packet.number, " SCTP session number ", packet.sctp.data_sid)
                        att_att_list.append(1)
        if (packet.s1ap.procedurecode == '13'):
                #С помощью фильтра оставляем лишь те пакеты, что находятся в той же sctp сессии, что и Auth. Request
                if (packet.sctp.data_sid == get_value):
                    #Преобразуем содержимое пакета в словарь, потом в список
                    get_dict1 = dict(packet.s1ap._all_fields)
                    keys1 = list(get_dict1.items())
                    #Делаем перебор всех значений, чтобы найти пакеты содержащие сообщ. Auth. Response
                    for o in keys1:
                        if (o == ('nas-eps.nas_msg_emm_type', '0x43')):
                            print("Attach complete was detected in ", packet.number, " SCTP session number ", packet.sctp.data_sid)
                            att_com_list.append(1)

Attach Request was detected in  1  SCTP session number  0x0004
Attach complete was detected in  29  SCTP session number  0x0004
Attach Request was detected in  65  SCTP session number  0x0006
Attach complete was detected in  82  SCTP session number  0x0006
Attach Request was detected in  167  SCTP session number  0x0009
Attach complete was detected in  184  SCTP session number  0x0009


In [5]:
# Auth request
auth_req_list = []
auth_res_list = []
auth_fail_list = []
for packet in capture:
    if (packet.highest_layer == 'S1AP'):
        d = packet.s1ap._all_fields
        #Находим все пакеты с кодом процедуры DownlinkNASTransport
        if (packet.s1ap.procedurecode == '11'):
            #Получаем идентификатор транспортной сессии SCTP
            get_value = packet.sctp.data_sid
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if packet.sctp.data_sid == get_value:
                #Преобразуем пакет в словарь, затем в список
                get_dict = dict(packet.s1ap._all_fields)
                keys = list(get_dict.items())
                #Делаем перебор всех значений в списке, чтобы найти пакеты, содержащие сообщение Authentication Request
                for i in keys:
                    if (i == ('nas-eps.nas_msg_emm_type', '0x52')):
                        #print("Auth request was detected in ", packet.number, " SCTP session number ", packet.sctp.data_sid)
                        auth_req_list.append(1)
        #Находим все пакеты с кодом процедуры UplinkNASTransport
        if (packet.s1ap.procedurecode == '13'):
                #С помощью фильтра оставляем лишь те пакеты, что находятся в той же sctp сессии, что и Auth. Request
                if (packet.sctp.data_sid == get_value):
                    #Преобразуем содержимое пакета в словарь, потом в список
                    get_dict1 = dict(packet.s1ap._all_fields)
                    keys1 = list(get_dict1.items())
                    #Делаем перебор всех значений, чтобы найти пакеты содержащие сообщ. Auth. Response
                    for o in keys1:
                        if (o == ('nas-eps.nas_msg_emm_type', '0x53')):
                            #print("Auth response was detected in ", packet.number, " SCTP session number", packet.sctp.data_sid)
                            auth_res_list.append(1)
        #Находим все пакеты, в которых процедура аутентификации законочилась неуспешно
        if (packet.s1ap.procedurecode == '13'):
            if (packet.sctp.data_sid == get_value):
                get_dict2 = dict(packet.s1ap._all_fields)
                keys2 = list(get_dict2.items())
                for p in keys2:
                    if (p == ('nas-eps.nas_msg_emm_type', '0x5c')):
                        #print("Auth failure was detected in ", packet.number, " SCTP session number", packet.sctp.data_sid)
                        auth_fail_list.append(1)

In [6]:
# Security Mode
sec_m_req = []
sec_m_res = []
for packet in capture:
    if (packet.highest_layer == 'S1AP'):
        d = packet.s1ap._all_fields
        #Находим все пакеты с кодом процедуры DownlinkNASTransport
        if (packet.s1ap.procedurecode == '11'):
            #Получаем идентификатор транспортной сессии SCTP
            get_value = packet.sctp.data_sid
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if packet.sctp.data_sid == get_value:
                #Преобразуем пакет в словарь, затем в список
                get_dict = dict(packet.s1ap._all_fields)
                keys = list(get_dict.items())
                for i in keys:
                    if (i == ('nas-eps.nas_msg_emm_type', '0x5d')):
                        #print("Security mode command detected in ", packet.number, " SCTP session ID", packet.sctp.data_sid)
                        sec_m_req.append(1)
        if (packet.s1ap.procedurecode == '13'):
                if (packet.sctp.data_sid == get_value):
                    get_dict1 = dict(packet.s1ap._all_fields)
                    keys1 = list(get_dict1.items())
                    for o in keys1:
                        if (o == ('nas-eps.nas_msg_emm_type', '0x5e')):
                            #print("Security mode complete detected in ", packet.number, " SCTP session ID", packet.sctp.data_sid)
                            sec_m_res.append(1)

In [7]:
# AIR
air_list = []
aia_list = []
for packet in capture:
    if (packet.highest_layer == 'DIAMETER'):
        d = packet.diameter._all_fields
        #Находим все пакеты с кодом процедуры DownlinkNASTransport
        if (packet.diameter.cmd_code == '318'):
            #Получаем идентификатор транспортной сессии SCTP
            get_value = packet.sctp.data_sid
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if packet.sctp.data_sid == get_value:
                #Преобразуем пакет в словарь, затем в список
                get_dict = dict(packet.diameter._all_fields)
                keys = list(get_dict.items())
                for i in keys:
                    if (i == ('diameter.AIR-Flags', '1')):
                        session_id = packet.diameter.session_id
                        #print("AIR detected in ", packet.number, " SCTP session ID", packet.diameter.session_id)
                        air_list.append(1)
        if (packet.diameter.cmd_code == '318'):
                if (packet.sctp.data_sid == get_value):
                    if (packet.diameter.session_id == session_id):
                        get_dict1 = dict(packet.diameter._all_fields)
                        keys1 = list(get_dict1.items())
                        for o in keys1:
                            if (o == ('diameter.Result-Code', '2001')):
                                #print("AIA ", packet.number, " SCTP session ID", packet.diameter.session_id)
                                aia_list.append(1)

In [8]:
# Create Session Request
cs_req_list = []
cs_res_list = []
for packet in capture:
    if (packet.highest_layer == 'GTPV2'):
        d = packet.gtpv2._all_fields
        #Находим все пакеты с кодом процедуры Create Session Request
        if (packet.gtpv2.message_type == '32'):
            #Получаем идентификатор сессии в GTPv2
            get_value = packet.gtpv2.seq
            get_s11teid = packet.gtpv2.f_teid_gre_key
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if (packet.gtpv2.seq == get_value):
                #print("CSRequest detected in ", packet.number, " Seq number ", packet.gtpv2.seq)
                #print("CSRequest was with S11 MME TEID ", packet.gtpv2.f_teid_gre_key)
                cs_req_list.append(1)
        if (packet.gtpv2.message_type == '33'):
                if (packet.gtpv2.seq == get_value):
                    if (packet.gtpv2.teid == get_s11teid):
                        #print("CSResponse detected in ", packet.number, " Seq Num ", packet.gtpv2.seq)
                        #print("CSResponse was with S11 SGW TIED ", packet.gtpv2.f_teid_gre_key)
                        cs_res_list.append(1)

In [9]:
#Modify Bearer Request
mb_req_list = []
mb_res_list = []
for packet in capture:
    if (packet.highest_layer == 'GTPV2'):
        d = packet.gtpv2._all_fields
        #Находим все пакеты с кодом процедуры Create Session Request
        if (packet.gtpv2.message_type == '34'):
            #Получаем идентификатор сессии в GTPv2
            get_value = packet.gtpv2.seq
            get_s11teid = packet.gtpv2.f_teid_gre_key
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if (packet.gtpv2.seq == get_value):
                #print("MBRequest detected in ", packet.number, " Seq number ", packet.gtpv2.seq)
                #print("MBRequest was with S11 MME TEID ", packet.gtpv2.f_teid_gre_key)
                mb_req_list.append(1)
        if (packet.gtpv2.message_type == '35'):
                if (packet.gtpv2.seq == get_value):
                    if (packet.gtpv2.teid == get_s11teid):
                        #print("MBResponse detected in ", packet.number, " Seq Num ", packet.gtpv2.seq)
                        #print("MBResponse was with S11 SGW TIED ", packet.gtpv2.f_teid_gre_key)
                        mb_res_list.append(1)

In [10]:
#Release Access Bearer Request
release_bear_req_list = []
release_bear_res_list = []
for packet in capture:
    if (packet.highest_layer == 'GTPV2'):
        d = packet.gtpv2._all_fields
        #Находим все пакеты с кодом процедуры Create Session Request
        if (packet.gtpv2.message_type == '170'):
            #Получаем идентификатор сессии в GTPv2
            get_value = packet.gtpv2.seq
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if (packet.gtpv2.seq == get_value):
                #print("Release Access Bearers Request detected in ", packet.number, " Seq number ", packet.gtpv2.seq)
                #print("Release Access Bearers Request with TEID ", packet.gtpv2.teid)
                release_bear_req_list.append(1)
        if (packet.gtpv2.message_type == '171'):
                if (packet.gtpv2.seq == get_value):
                    #print("Release Access Bearers Response detected in ", packet.number, " Seq Num ", packet.gtpv2.seq)
                    #print("Release Access Bearers Response was with TIED ", packet.gtpv2.teid)
                    release_bear_res_list.append(1)

In [11]:
#Delete Bearer Request
db_req_list = []
db_res_list = []
for packet in capture:
    if (packet.highest_layer == 'GTPV2'):
        d = packet.gtpv2._all_fields
        #Находим все пакеты с кодом процедуры Create Session Request
        if (packet.gtpv2.message_type == '99'):
            #Получаем идентификатор сессии в GTPv2
            get_value = packet.gtpv2.seq
            #Фильтруем пакеты в рамкой одной транспортной сессии
            if (packet.gtpv2.seq == get_value):
                #print("Delete Bearer Request detected in ", packet.number, " Seq number ", packet.gtpv2.seq)
                #print("Delete Bearer Request with TEID ", packet.gtpv2.teid)
                db_req_list.append(1)
        if (packet.gtpv2.message_type == '100'):
                if (packet.gtpv2.seq == get_value):
                    #print("Delete Bearer Response detected in ", packet.number, " Seq Num ", packet.gtpv2.seq)
                    #print("Delete Bearer Response was with TIED ", packet.gtpv2.teid)
                    db_res_list.append(1)

In [28]:
if (len(att_att_list) > 0) and ( len(att_com_list) > 0):
    print("Attach Request/Complete Test Passed")
    print("There are ", len(att_att_list), " attempt ", " and ", len(att_com_list), " complete")

Attach Request/Complete Test Passed
There are  3  attempt   and  3  complete


In [29]:
if (len(auth_req_list) > 0) and ( len(auth_res_list) > 0):
    print("Auth Request/Response Test Passed")
    print("There are ", len(auth_req_list), " request ", " and ", len(auth_res_list), " response")

Auth Request/Response Test Passed
There are  4  request   and  3  response


In [30]:
if (len(sec_m_req) > 0) and ( len(sec_m_res) > 0):
    print("Security Mode/Complete Test Passed")
    print("There are ", len(sec_m_req), " command ", " and ", len(sec_m_res), " complete")

Security Mode/Complete Test Passed
There are  3  command   and  3  complete


In [31]:
if (len(air_list) > 0) and ( len(aia_list) > 0):
    print("AIR/AIA Test Passed")
    print("There are ", len(air_list), " AIR ", " and ", len(aia_list), " AIA")

AIR/AIA Test Passed
There are  4  AIR   and  4  AIA


In [32]:
if (len(cs_req_list) > 0) and ( len(cs_res_list) > 0):
    print("Create Session Request Test Passed")
    print("There are ", (len(cs_req_list)/2), " session request ", " and ", len(cs_res_list), " session respone")

Create Session Request Test Passed
There are  9.0  session request   and  9  session respone


In [33]:
if (len(mb_req_list) > 0) and ( len(mb_res_list) > 0):
    print("Modify Bearer Request/Response Test Passed")
    print("There are ", (len(mb_req_list)/2), " modify request ", " and ", len(mb_res_list), " modify respone")

Modify Bearer Request/Response Test Passed
There are  13.0  modify request   and  11  modify respone


In [36]:
if (len(release_bear_req_list) > 0) and ( len(release_bear_res_list) > 0):
    print("Release Bearer Test Passed")
    print("There are ", (len(release_bear_req_list)), " release request ", " and ", len(release_bear_res_list), " release respone")

Release Bearer Test Passed
There are  6  release request   and  2  release respone


In [37]:
if (len(db_req_list) > 0) and ( len(db_res_list) > 0):
    print("Delete Bearer Request/Response Test Passed")
    print("There are ", (len(db_req_list)), " delete request ", " and ", len(db_res_list), " delete respone")

Delete Bearer Request/Response Test Passed
There are  3  delete request   and  4  delete respone
